In [22]:
import numpy as np
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score
from textblob import TextBlob

# Vader Score

In [23]:
df = pd.read_csv('Data/iee_data.csv')

df.head()

,id,created_at,text,sentiment,label
0,77522,2020-04-15 01:03:46+00:00,"RT @RobertBeadles: Yo💥\nEnter to WIN 1,000 Mon...",positive,1
1,661634,2020-06-25 06:20:06+00:00,#SriLanka surcharge on fuel removed!\n⛽📉\nThe ...,negative,-1
2,413231,2020-06-04 15:41:45+00:00,Net issuance increases to fund fiscal programs...,positive,1
3,760262,2020-07-03 19:39:35+00:00,RT @bentboolean: How much of Amazon's traffic ...,positive,1
4,830153,2020-07-09 14:39:14+00:00,$AMD Ryzen 4000 desktop CPUs looking ‘great’ a...,positive,1


In [24]:
analyzer = SentimentIntensityAnalyzer()

In [25]:
# calculate score
def get_vader_score(sentence): 
    compound=analyzer.polarity_scores(sentence)['compound']
    if compound >= 0.05: 
        return 1
    elif (compound > -0.05) and (compound <0.05): 
        return 0
    if compound <= 0.05:
        return -1
    
df['vader']=df.apply(lambda x: get_vader_score(x['text']), axis=1)

In [26]:
df.head()

,id,created_at,text,sentiment,label,vader
0,77522,2020-04-15 01:03:46+00:00,"RT @RobertBeadles: Yo💥\nEnter to WIN 1,000 Mon...",positive,1,1
1,661634,2020-06-25 06:20:06+00:00,#SriLanka surcharge on fuel removed!\n⛽📉\nThe ...,negative,-1,1
2,413231,2020-06-04 15:41:45+00:00,Net issuance increases to fund fiscal programs...,positive,1,-1
3,760262,2020-07-03 19:39:35+00:00,RT @bentboolean: How much of Amazon's traffic ...,positive,1,1
4,830153,2020-07-09 14:39:14+00:00,$AMD Ryzen 4000 desktop CPUs looking ‘great’ a...,positive,1,0


In [27]:
print(f'Accuracy: {accuracy_score(df.dropna()["label"].values, df.dropna()["vader"].values)}')

Accuracy: 0.6661538461538462


# TextBlob

In [28]:
scores_blob = []

# Declare variables for scores
polarity = []
subjectivity = []

for i in range(df['text'].shape[0]):
    polarity = TextBlob(df['text'][i]).sentiment[0]
    subjectivity = TextBlob(df['text'][i]).sentiment[1]
    
    scores_blob.append({"Polarity_blob": polarity,
                       "Subjectivity_blob": subjectivity
                  })

In [29]:
sentiments_score_blob = pd.DataFrame.from_dict(scores_blob)
df = df.join(sentiments_score_blob)

def getAnalysis(score):
    if score >= 0.05: 
        return 1
    elif (score > -0.05) and (score <0.05): 
        return 0
    if score <= 0.05:
        return -1
    
df['blob'] = df['Polarity_blob'].apply(getAnalysis)

In [30]:
print(f'Accuracy: {accuracy_score(df.dropna()["label"].values, df.dropna()["blob"].values)}')

Accuracy: 0.5261538461538462


# SentiWordNet